In [115]:
import os
import json
import pandas as pd
import traceback

In [116]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain,SequentialChain
from langchain.callbacks.base import BaseCallbackHandler
from langchain.callbacks import get_openai_callback
import PyPDF2

In [117]:
from dotenv import load_dotenv
load_dotenv()
key=os.getenv("GEMINI_KEY")

In [118]:
client=ChatGoogleGenerativeAI(
    model="gemma-3-12b-it",
    google_api_key=key,
    temperature=0.5
)

In [191]:
TEMPLATE = """
You are an expert MCQ generator.  
Create {number} MCQs from {text} for {subject} students in a {tone} tone.  

Rules:
- Output ONLY valid JSON.  
- Do NOT add ``` or any markdown formatting.  
- Do NOT write the word 'json'.  
- Return pure JSON object only.  

Here is the RESPONSE_JSON format you must strictly follow:  
{response_json}
"""


In [192]:
template2="""
you are an expert english grammarian and writer who evaluate  the quiz for {subject} students.fix those quick _mcq:
{quiz} exatly please dont use any markdown make it suitable to load in json.loads
check from an expert english writer of the above quiz
"""

In [ ]:
quiz_generation=PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)
quiz_evalute=PromptTemplate(
    input_variables=['subject','quiz'],
    template=template2
)

In [195]:
text="A semiconductor is a material whose electrical conductivity lies between that of a conductor (like copper) and an insulator (like rubber). The most widely used semiconductor is silicon, though germanium and gallium arsenide are also common.In semiconductors, the flow of current depends on the movement of charge carriers—electrons and holes. At absolute zero, a pure semiconductor behaves like an insulator because no free charge carriers are present. However, as temperature increases, some electrons gain energy and jump from the valence band to the conduction band, leaving behind holes. This creates a limited but controllable flow of current.To enhance conductivity, semiconductors are doped with small amounts of impurities. Adding a pentavalent impurity (like phosphorus) produces an n-type semiconductor with extra electrons as majority carriers. Adding a trivalent impurity (like boron) produces a p-type semiconductor, where holes dominate.The junction of p-type and n-type materials forms a p–n junction, the basis of diodes, transistors, solar cells, and integrated circuits. By controlling electron and hole movement, semiconductors enable switching, amplification, and energy conversion.Thus, semiconductors form the backbone of modern electronics, powering everything from computers and smartphones to LEDs and microchips."

In [196]:
response_json={
    "1":{
        "mcq":"multiple choice quesiton",
        "option":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    "2":{
        "mcq":"multiple choice quesiton",
        "option":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    "3":{
        "mcq":"multiple choice quesiton",
        "option":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
}

In [ ]:
quiz_chain=LLMChain(llm=client,prompt=quiz_generation,output_key="quiz",verbose=True)
review_chain=LLMChain(llm=client,prompt=quiz_evalute,output_key="review",verbose=True)

In [199]:
final_chain=SequentialChain(chains=[quiz_chain,review_chain],input_variables=["text","number","subject","tone","response_json"],
                            output_variables=['quiz',"review"],verbose=True
                            )

In [200]:
with open(r"C:\Users\Avijit\Desktop\MCQ_GENERATOR\data.txt",'r') as file:
    TEXT=file.read()

In [213]:
NUMBER=51,
SUBJECT="PHYSICS",
TONE='SIMPLE'

In [214]:
with get_openai_callback() as cb:
    response = final_chain.invoke({
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(response_json)
    })



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert MCQ generator.  
Create (51,) MCQs from A semiconductor is a material whose electrical conductivity lies between that of a conductor (like copper) and an insulator (like rubber). The most widely used semiconductor is silicon, though germanium and gallium arsenide are also common.
In semiconductors, the flow of current depends on the movement of charge carriers—electrons and holes. At absolute zero, a pure semiconductor behaves like an insulator because no free charge carriers are present. However, as temperature increases, some electrons gain energy and jump from the valence band to the conduction band, leaving behind holes. This creates a limited but controllable flow of current.
To enhance conductivity, semiconductors are doped with small amounts of impurities. Adding a pentavalent impurity (like phosphorus) produces an n-type semiconductor with extra electrons as 

In [215]:
print(f"TOTAL TOKENS {cb.total_tokens}")
print(f"TOTAL COST {cb.total_cost}")
print(f"COMPLETION TOKEN {cb.completion_tokens}")
print(f"PROMPT TOKEN {cb.prompt_tokens}")

TOTAL TOKENS 4280
TOTAL COST 0.0
COMPLETION TOKEN 0
PROMPT TOKEN 4280


In [216]:
quiz=response.get("quiz")

In [217]:
json.loads(quiz)

{'1': {'mcq': 'What is a semiconductor?',
  'option': {'a': 'A material that conducts electricity very well.',
   'b': 'A material with conductivity between a conductor and an insulator.',
   'c': 'A material that completely blocks electrical current.',
   'd': 'A type of metal used in electronics.'},
  'correct': 'b'},
 '2': {'mcq': 'Which of the following is the most widely used semiconductor?',
  'option': {'a': 'Germanium',
   'b': 'Gallium Arsenide',
   'c': 'Silicon',
   'd': 'Copper'},
  'correct': 'c'},
 '3': {'mcq': 'What happens to a pure semiconductor at absolute zero?',
  'option': {'a': 'It conducts electricity very well.',
   'b': 'It behaves like an insulator.',
   'c': 'It becomes a superconductor.',
   'd': 'It emits light.'},
  'correct': 'b'},
 '4': {'mcq': 'What creates charge carriers in a semiconductor as temperature increases?',
  'option': {'a': 'The addition of impurities.',
   'b': 'Electrons gaining energy and jumping bands.',
   'c': 'A decrease in resistanc

In [218]:

def quiz_json_to_csv(quiz_json: str, csv_filename: str = "quiz.csv"):
    """
    Convert quiz JSON (with numbered keys) to a CSV file.
    
    :param quiz_json: JSON string containing quiz data
    :param csv_filename: Output CSV file name
    """
    # Load JSON string into dict
    quiz_dict = json.loads(quiz_json)
    
    # Open CSV file for writing
    with open(csv_filename, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        
        # Write header
        writer.writerow(["Q.No", "Question", "Options", "Correct Answer"])
        
        # Write each question
        for q_no, q_data in quiz_dict.items():
            question = q_data["mcq"]
            options = "; ".join([f"{k}: {v}" for k, v in q_data["option"].items()])
            correct = q_data["correct"]
            writer.writerow([q_no, question, options, correct])
    
    print(f"✅ Quiz saved to {csv_filename}")


In [219]:
quiz_json_to_csv(quiz, "quiz1.csv")


✅ Quiz saved to quiz1.csv


In [212]:
pd.read_csv(r"C:\Users\Avijit\Desktop\MCQ_GENERATOR\experiment\quiz.csv")

,Q.No,Question,Options,Correct Answer
0,1,What is a semiconductor?,a: A material that conducts electricity very w...,b
1,2,What happens to a pure semiconductor at absolu...,a: It conducts electricity very well.; b: It b...,b
2,3,What is doping in semiconductors?,a: Heating the semiconductor to a high tempera...,b
3,4,What is the primary function of a diode?,a: To amplify electrical signals.; b: To allow...,b
4,5,What is rectification?,a: Converting AC to DC.; b: Increasing voltage...,a
